# Bouw een classificatiemodel: Heerlijke Aziatische en Indiase Keukens


## Categorieclassificaties 2

In deze tweede les over classificatie gaan we `meer manieren` verkennen om categorische data te classificeren. We zullen ook leren over de gevolgen van het kiezen van de ene classifier boven de andere.

### [**Pre-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Vereisten**

We gaan ervan uit dat je de vorige lessen hebt afgerond, aangezien we enkele eerder geleerde concepten zullen voortzetten.

Voor deze les hebben we de volgende pakketten nodig:

-   `tidyverse`: Het [tidyverse](https://www.tidyverse.org/) is een [verzameling van R-pakketten](https://www.tidyverse.org/packages) die datawetenschap sneller, eenvoudiger en leuker maakt!

-   `tidymodels`: Het [tidymodels](https://www.tidymodels.org/) framework is een [verzameling van pakketten](https://www.tidymodels.org/packages/) voor modellering en machine learning.

-   `themis`: Het [themis-pakket](https://themis.tidymodels.org/) biedt extra stappen in recepten om om te gaan met onevenwichtige data.

Je kunt ze installeren met:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Als alternatief controleert het onderstaande script of je de benodigde pakketten hebt om deze module te voltooien en installeert ze voor je als ze ontbreken.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Een classificatiekaart**

In onze [vorige les](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) probeerden we de vraag te beantwoorden: hoe kiezen we tussen meerdere modellen? Voor een groot deel hangt dit af van de kenmerken van de data en het type probleem dat we willen oplossen (bijvoorbeeld classificatie of regressie?).

Eerder hebben we geleerd over de verschillende opties die je hebt bij het classificeren van data met behulp van Microsoft's cheat sheet. Het Machine Learning-framework van Python, Scikit-learn, biedt een vergelijkbare maar meer gedetailleerde cheat sheet die verder kan helpen om je keuze van estimators (een andere term voor classifiers) te verfijnen:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tip: [bekijk deze kaart online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) en klik langs het pad om de documentatie te lezen.
>
> De [Tidymodels referentiesite](https://www.tidymodels.org/find/parsnip/#models) biedt ook uitstekende documentatie over verschillende soorten modellen.

### **Het plan** 🗺️

Deze kaart is erg handig zodra je een goed begrip hebt van je data, omdat je langs de paden kunt 'wandelen' naar een beslissing:

-   We hebben \>50 samples

-   We willen een categorie voorspellen

-   We hebben gelabelde data

-   We hebben minder dan 100K samples

-   ✨ We kunnen een Linear SVC kiezen

-   Als dat niet werkt, aangezien we numerieke data hebben

    -   Kunnen we een ✨ KNeighbors Classifier proberen

        -   Als dat niet werkt, probeer ✨ SVC en ✨ Ensemble Classifiers

Dit is een erg handig pad om te volgen. Laten we nu meteen aan de slag gaan met het [tidymodels](https://www.tidymodels.org/) modellering framework: een consistente en flexibele verzameling R-pakketten die is ontwikkeld om goede statistische praktijken te bevorderen 😊.

## 2. Splits de data en ga om met een onevenwichtig data set.

Uit onze vorige lessen hebben we geleerd dat er een set van gemeenschappelijke ingrediënten was in onze keukens. Ook was er een behoorlijk ongelijke verdeling in het aantal keukens.

We gaan hiermee om door:

-   De meest voorkomende ingrediënten die verwarring veroorzaken tussen verschillende keukens te verwijderen, met behulp van `dplyr::select()`.

-   Een `recipe` te gebruiken die de data preprocessen om deze klaar te maken voor modellering door een `over-sampling` algoritme toe te passen.

We hebben het bovenstaande al behandeld in de vorige les, dus dit zou een eitje moeten zijn 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Omgaan met onevenwichtige data

Onevenwichtige data heeft vaak een negatieve invloed op de modelprestaties. Veel modellen presteren het beste wanneer het aantal observaties gelijk is en hebben daardoor moeite met onevenwichtige data.

Er zijn grofweg twee manieren om met onevenwichtige datasets om te gaan:

-   het toevoegen van observaties aan de minderheidsklasse: `Over-sampling`, bijvoorbeeld met behulp van een SMOTE-algoritme, dat synthetisch nieuwe voorbeelden van de minderheidsklasse genereert door gebruik te maken van de dichtstbijzijnde buren van deze gevallen.

-   het verwijderen van observaties uit de meerderheidsklasse: `Under-sampling`

In onze vorige les hebben we gedemonstreerd hoe je met onevenwichtige datasets kunt omgaan met behulp van een `recipe`. Een recipe kun je zien als een blauwdruk die beschrijft welke stappen op een dataset moeten worden toegepast om deze klaar te maken voor data-analyse. In ons geval willen we een gelijke verdeling in het aantal keukens in onze `training set`. Laten we meteen aan de slag gaan.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nu zijn we klaar om modellen te trainen 👩‍💻👨‍💻!

## 3. Verder dan multinomiale regressiemodellen

In onze vorige les hebben we gekeken naar multinomiale regressiemodellen. Laten we enkele flexibelere modellen voor classificatie verkennen.

### Support Vector Machines

In de context van classificatie is `Support Vector Machines` een machine learning-techniek die probeert een *hypervlak* te vinden dat de klassen op de "beste" manier scheidt. Laten we een eenvoudig voorbeeld bekijken:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ scheidt de klassen niet. H2~ doet dat wel, maar slechts met een kleine marge. H3~ scheidt ze met de maximale marge.

#### Lineaire Support Vector Classifier

Support-Vector clustering (SVC) is een onderdeel van de Support-Vector machines familie van ML-technieken. Bij SVC wordt het hypervlak zo gekozen dat het `de meeste` trainingsobservaties correct scheidt, maar `kan enkele observaties verkeerd classificeren`. Door toe te staan dat sommige punten aan de verkeerde kant liggen, wordt de SVM robuuster tegen uitschieters en daardoor beter in het generaliseren naar nieuwe data. De parameter die deze overtreding reguleert wordt aangeduid als `cost`, welke een standaardwaarde heeft van 1 (zie `help("svm_poly")`).

Laten we een lineaire SVC maken door `degree = 1` in te stellen in een polynomiaal SVM-model.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nu we de stappen voor voorbewerking en modelspecificatie hebben vastgelegd in een *workflow*, kunnen we doorgaan met het trainen van de lineaire SVC en tegelijkertijd de resultaten evalueren. Voor prestatiemetrics laten we een set met metrics maken die de volgende aspecten evalueert: `accuracy`, `sensitivity`, `Positive Predicted Value` en `F Measure`.

> `augment()` voegt kolom(men) toe met voorspellingen aan de gegeven data.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

De support vector machine (SVM) is een uitbreiding van de support vector classifier om een niet-lineaire grens tussen de klassen mogelijk te maken. In essentie gebruiken SVM's de *kerneltruc* om de feature space uit te breiden en zich aan te passen aan niet-lineaire relaties tussen klassen. Een populaire en zeer flexibele kernelfunctie die door SVM's wordt gebruikt, is de *Radiale basisfunctie.* Laten we eens kijken hoe deze presteert op onze data.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Veel beter 🤩!

> ✅ Zie alstublieft:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> voor verdere verdieping.

### Nabijste buur-classificatoren

*K*-nearest neighbor (KNN) is een algoritme waarbij elke observatie wordt voorspeld op basis van de *overeenkomst* met andere observaties.

Laten we er een toepassen op onze data.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Het lijkt erop dat dit model niet zo goed presteert. Waarschijnlijk zal het aanpassen van de argumenten van het model (zie `help("nearest_neighbor")`) de prestaties verbeteren. Zorg ervoor dat je dit uitprobeert.

> ✅ Zie:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> om meer te leren over *K*-Nearest Neighbors classifiers.

### Ensemble classifiers

Ensemble-algoritmen werken door meerdere basismodellen te combineren om een optimaal model te creëren, op een van de volgende manieren:

`bagging`: toepassen van een *gemiddelde functie* op een verzameling basismodellen

`boosting`: bouwen van een reeks modellen die op elkaar voortbouwen om de voorspellende prestaties te verbeteren.

Laten we beginnen met het uitproberen van een Random Forest-model, dat een grote verzameling beslissingsbomen bouwt en vervolgens een gemiddelde functie toepast voor een beter algemeen model.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Goed gedaan 👏!

Laten we ook experimenteren met een Boosted Tree-model.

Boosted Tree definieert een ensemblemethode die een reeks opeenvolgende beslissingsbomen creëert, waarbij elke boom afhankelijk is van de resultaten van eerdere bomen om stapsgewijs de fout te verminderen. Het richt zich op de gewichten van verkeerd geclassificeerde items en past de fit van de volgende classifier aan om dit te corrigeren.

Er zijn verschillende manieren om dit model te fitten (zie `help("boost_tree")`). In dit voorbeeld passen we Boosted Trees toe via de `xgboost` engine.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Zie:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Verkent het AdaBoost-model, een goed alternatief voor xgboost.
>
> om meer te leren over Ensemble classifiers.

## 4. Extra - meerdere modellen vergelijken

We hebben in dit lab behoorlijk wat modellen gefit 🙌. Het kan tijdrovend of lastig worden om veel workflows te maken met verschillende sets preprocessors en/of modelspecificaties en vervolgens één voor één de prestatiestatistieken te berekenen.

Laten we kijken of we dit kunnen aanpakken door een functie te maken die een lijst van workflows op de trainingsset fit en vervolgens de prestatiestatistieken retourneert op basis van de testset. We gaan `map()` en `map_dfr()` uit het [purrr](https://purrr.tidyverse.org/) pakket gebruiken om functies toe te passen op elk element in een lijst.

> [`map()`](https://purrr.tidyverse.org/reference/map.html)-functies stellen je in staat om veel for-loops te vervangen door code die zowel beknopter als makkelijker te lezen is. De beste plek om meer te leren over de [`map()`](https://purrr.tidyverse.org/reference/map.html)-functies is het [iteratiehoofdstuk](http://r4ds.had.co.nz/iteration.html) in R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) stelt gebruikers in staat om een groot aantal modellen te maken en eenvoudig te trainen, maar is voornamelijk ontworpen om te werken met hersteekproeftechnieken zoals `cross-validation`, een aanpak die we nog moeten behandelen.

## **🚀Uitdaging**

Elk van deze technieken heeft een groot aantal parameters die je kunt aanpassen, zoals `cost` in SVMs, `neighbors` in KNN, `mtry` (willekeurig geselecteerde voorspellers) in Random Forest.

Onderzoek de standaardparameters van elk model en denk na over wat het aanpassen van deze parameters zou betekenen voor de kwaliteit van het model.

Om meer te weten te komen over een specifiek model en zijn parameters, gebruik: `help("model")`, bijvoorbeeld `help("rand_forest")`.

> In de praktijk *schatten* we meestal de *beste waarden* voor deze parameters door veel modellen te trainen op een `gesimuleerd gegevensset` en te meten hoe goed al deze modellen presteren. Dit proces wordt **tuning** genoemd.

### [**Quiz na de les**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Herziening & Zelfstudie**

Er wordt veel vakjargon gebruikt in deze lessen, dus neem even de tijd om [deze lijst](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) met nuttige terminologie door te nemen!

#### DANK AAN:

[`Allison Horst`](https://twitter.com/allison_horst/) voor het maken van de geweldige illustraties die R toegankelijker en aantrekkelijker maken. Vind meer illustraties in haar [galerij](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) en [Jen Looper](https://www.twitter.com/jenlooper) voor het maken van de originele Python-versie van deze module ♥️

Veel leerplezier,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Illustratie door @allison_horst</figcaption>



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, willen we u erop wijzen dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
